In [ ]:
from pathlib import Path

import pandas as pd

In [ ]:
datadf = pd.read_csv("../data/the_rise_of_healthcare_jobs_disclosed_data_by_msa.csv")
datadf.head(10)

In [ ]:
datadf["metro13"].unique()

In [ ]:
import json

import altair as alt
import pandas as pd

# import natsort
alt.data_transformers.enable("vegafusion")

In [ ]:
jsonFile = json.load(Path.open("../data/2021_US_CBSA.geojson"))
cbsa = pd.DataFrame(jsonFile["features"])
cbsa["Location Name"] = cbsa["properties"].apply(lambda x: x["NAME"])
cbsa["Location ID"] = cbsa["properties"].apply(lambda x: x["CBSAFP"])
cbsa = cbsa[~cbsa["Location Name"].str.contains("HI|AK|PR", regex=True)]

In [ ]:
jsonFile = json.load(Path.open("../data/2021_US_States.geojson"))
states = pd.DataFrame(jsonFile["features"])
states["State Name"] = states["properties"].apply(lambda x: x["NAME"])
states["State Abbreviation"] = states["properties"].apply(lambda x: x["STUSPS"])
states = states[
    ~states["State Abbreviation"].isin(["HI", "AK", "PR", "VI", "GU", "MP", "AS"])
]

In [ ]:
# Choropleth Base Map
alt.Chart(cbsa, title="CBSA").mark_geoshape(stroke="black", strokeWidth=0.75).encode(
    color=alt.value("lightgrey"),
    tooltip=[
        alt.Tooltip("Location Name:N", title="Location Name:"),
        alt.Tooltip("Location ID:N", title="Location ID:"),
    ],
).project(type="albers").properties(width=1000, height=800)

In [ ]:
# Choropleth Base Map
alt.Chart(states, title="U.S. States").mark_geoshape(
    stroke="black", strokeWidth=0.75
).encode(
    color=alt.value("lightgrey"),
    tooltip=[alt.Tooltip("State Name:N", title="State Name:")],
).project(type="albers").properties(width=1000, height=800)

In [ ]:
# Reshape df to long format
value_cols = [c for c in datadf.columns if c not in ["metro13", "metro_title"]]
df_long = datadf.melt(
    id_vars=["metro13", "metro_title"],
    value_vars=value_cols,
    var_name="indicator",
    value_name="value",
)

indicator_dropdown = alt.binding_select(options=value_cols, name="Select variable: ")
indicator_selection = alt.selection_point(fields=["indicator"], bind=indicator_dropdown)
point_selection = alt.selection_point(fields=["metro_title"])

base = (
    alt.Chart(states)
    .mark_geoshape(fill="lightgray", stroke="black", strokeWidth=0.5)
    .encode(tooltip=alt.Tooltip("State Name:N", title="State:"))
)

cbsa_layer = (
    alt.Chart(df_long)
    .mark_geoshape(stroke="#888888", strokeWidth=0.8)
    .encode(
        color=alt.Color(
            "value:Q",
            scale=alt.Scale(scheme="viridis"),
            legend=alt.Legend(orient="bottom"),
        ),
        opacity=alt.condition(point_selection, alt.value(1.0), alt.value(0.25)),
        tooltip=[
            alt.Tooltip("metro_title:N", title="Metropolitan Area"),
            alt.Tooltip("indicator:N", title="Variable"),
            alt.Tooltip("value:Q", title="Value"),
        ],
    )
    .transform_lookup(
        lookup="metro13",
        from_=alt.LookupData(
            cbsa, key="Location ID", fields=["geometry", "type", "Location Name"]
        ),
    )
    # .add_params(indicator_selection)
    .transform_filter(indicator_selection)
    # .project(type="albers")
    # .properties(width=900, height=600, title="CBSA-Level Data by Indicator")
)

state_layer = (
    alt.Chart(df_long)
    .mark_geoshape(stroke="#414141", strokeWidth=0.6)
    .encode(
        color=alt.Color(
            "value:Q",
            scale=alt.Scale(scheme="viridis"),
            legend=alt.Legend(orient="bottom"),
        ),
        opacity=alt.condition(point_selection, alt.value(1.0), alt.value(0.25)),
        tooltip=[
            alt.Tooltip("metro_title:N", title="State"),
            alt.Tooltip("indicator:N", title="Variable"),
            alt.Tooltip("value:Q", title="Value"),
        ],
    )
    .transform_lookup(
        lookup="metro_title",
        from_=alt.LookupData(
            states,
            key="State Abbreviation",
            fields=["geometry", "type", "Location Name"],
        ),
    )
    # .add_params(indicator_selection)
    .transform_filter(indicator_selection)
    # .project(type="albers")
    # .properties(width=900, height=600, title="State-Level Data by Indicator")
)

layered_map = (base + state_layer + cbsa_layer).add_params(
    indicator_selection, point_selection
)
layered_map = (
    layered_map.project(type="albers")
    .properties(width=1000, height=600, title="CBSA and State-Level Data by Indicator")
    .interactive()
)

# layered_map

In [ ]:
bar_chart = (
    alt.Chart(df_long, title="Values by Region")
    .mark_bar()
    .encode(
        x=alt.X(
            "metro_title:N",
            sort=alt.SortField(
                field="value",  # dynamically sort by value
                order="ascending",  # or 'descending' if you prefer
            ),
            title=None,
        ),
        y=alt.Y("value:Q", title="Value"),
        color=alt.condition(
            point_selection,
            alt.Color("value:Q", scale=alt.Scale(scheme="viridis")),
            alt.value("lightgray"),
        ),
        opacity=alt.condition(point_selection, alt.value(1.0), alt.value(0.6)),
        tooltip=[
            alt.Tooltip("metro_title:N", title="Region"),
            alt.Tooltip("value:Q", title="Value"),
        ],
    )
    .add_params(point_selection)
    .transform_filter(indicator_selection)
    .properties(width=1200, height=200)
)

layered_map & bar_chart

In [ ]:
import os

import plotly.express as px
import streamlit as st
from dotenv import load_dotenv

load_dotenv()

# Access your token
mapbox_token = os.getenv("MAPBOX_TOKEN")

px.set_mapbox_access_token(mapbox_token)

In [ ]:
# Streamlit dropdown
st.title("U.S. MSA Choropleth — Interactive Demo")

indicator = st.selectbox("Select variable:", sorted(df_long["indicator"].unique()))